In [1]:
import ee
import geemap
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import pandas as pd

In [2]:
print(ee.__version__)
print(geemap.__version__)
ee.Initialize()

0.1.249
0.8.8


## Load shape files as regions

In [3]:
manaus_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)
info = manaus.geometry()
manaus = ee.Geometry(info)

atto_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_atto_footprint/venus_atto_footprint.shp'
atto = geemap.shp_to_ee(atto_shp)
info = atto.geometry()
atto = ee.Geometry(info)

corumba_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_corumba_footprint/venus_corumba_footprint.shp'
corumba = geemap.shp_to_ee(corumba_shp)
info = corumba.geometry()
corumba = ee.Geometry(info)

mato_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_mato_footprint/venus_mato_footprint.shp'
mato = geemap.shp_to_ee(mato_shp)
info = mato.geometry()
mato = ee.Geometry(info)

saop_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_saop_footprint/venus_saop_footprint.shp'
saop = geemap.shp_to_ee(saop_shp)
info = saop.geometry()
saop = ee.Geometry(info)

parana_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_parana_footprint/venus_parana_footprint.shp'
parana = geemap.shp_to_ee(parana_shp)
info = parana.geometry()
parana = ee.Geometry(info)

comodo_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_comodo_footprint/venus_comodo_footprint.shp'
comodo = geemap.shp_to_ee(comodo_shp)
info = comodo.geometry()
comodo = ee.Geometry(info)

k34_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_k34_footprint/venus_k34_footprint.shp'
k34 = geemap.shp_to_ee(k34_shp)
info = k34.geometry()
k34 = ee.Geometry(info)

jam_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_jam_footprint/venus_jam_footprint.shp'
jam = geemap.shp_to_ee(jam_shp)
info = jam.geometry()
jam = ee.Geometry(info)

In [69]:
# Import grid
sitename = 
grid_shp = '/data/6ru/amazonas_grid.shp'
grid_fc = geemap.shp_to_ee(grid_shp)

# Extract coordinates from feature collection
grid_fc_dict = grid_fc.getInfo()
feats = grid_fc_dict['features']
coord_list = []
for d in feats:
    coords = d['geometry']['coordinates']
    coord_list.append(coords)

# Create polygons from coordinates
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)

# Select only the first 100 grid squares
idx = list(range(0,100))
polys = [ polys[i] for i in idx]

# Select one of first 100 grid squares
geom = polys[0]
fc = ee.FeatureCollection(geom)

## Define some processing routines 

In [70]:
# Mean
def region_mean_Image(img):
    # mean over all pixels in the region for the image
    return img.set('mean', img.reduceRegion(ee.Reducer.mean(), geometry=geom, scale=20)) 


def region_mean_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_mean_Image)
    stats_list = stats.aggregate_array('mean')
    return np.array(stats_list.getInfo())

# Median
def region_median_Image(img):
    # mean over all pixels in the region for the image
    return img.set('median', img.reduceRegion(ee.Reducer.median(), geometry=geom, scale=20)) 

def region_median_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_median_Image)
    stats_list = stats.aggregate_array('median')
    return np.array(stats_list.getInfo())

# Stddev
def region_stddev_Image(img):
    # stddev over all pixels in the region for the image
    return img.set('stddev', img.reduceRegion(ee.Reducer.stdDev(), geometry=geom, scale=20)) 

def region_stddev_ImageCollection(ic):
    # stddev over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_stddev_Image)
    stats_list = stats.aggregate_array('stddev')
    return np.array(stats_list.getInfo())

# Percentiles
def region_percentiles_Image(img):
    # 5,10,25,75,90,95 percentiles over all pixels in the region for the image
    return img.set('percentiles', img.reduceRegion(ee.Reducer.percentile([5,10,25,75,90,95]), geometry=fc, scale=20)) #) mean, median, stdDev, percentiles

def region_percentiles_ImageCollection(ic):
    # percentiles over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_percentiles_Image)
    stats_list = stats.aggregate_array('percentiles')
    return np.array(stats_list.getInfo())


In [71]:
#Append ndre percentile dict values to new lists
def p_list(alist, varname):
    p10, p25, p5, p75, p90, p95 = [], [], [], [], [], []
    for value in alist:
        p5.append(value[varname+'_p5'])
        p10.append(value[varname+'_p10'])
        p25.append(value[varname+'_p25'])
        p75.append(value[varname+'_p75'])
        p90.append(value[varname+'_p90'])
        p95.append(value[varname+'_p95'])
    return p5, p10, p25, p75, p90, p95

In [72]:
# get list of dates from ImageCollection
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

In [73]:
#Take only the ndre mean, med, stddev dict values and append to new list
def newList(alist, varname):
    list_name = []
    for value in alist:
        list_name.append(value[varname])
    return list_name

# ########################################
## ERA5
# ########################################

In [74]:
# Read the ERA5 data 
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY

SDATE = '2017-01-01'
EDATE = '2020-07-09'

# Daily mean 2m air temperature
era5_2mt = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                   .select('mean_2m_air_temperature')
                   .filterBounds(fc)
                   .filter(ee.Filter.date(SDATE, EDATE)))
#print(era5_2mt))

# Daily total precipitation sums
era5_tp = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                  .select('total_precipitation')
                  .filterBounds(fc)
                  .filter(ee.Filter.date(SDATE, EDATE)))

# Daily mean 2m dewpoint temperature
era5_2d = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                  .select('dewpoint_2m_temperature')
                  .filterBounds(fc)
                  .filter(ee.Filter.date(SDATE, EDATE)))

# Daily mean sea-level pressure
era5_mslp = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                    .select('mean_sea_level_pressure')
                    .filterBounds(fc)
                    .filter(ee.Filter.date(SDATE, EDATE)))
# Convert pressure levels from Pa to hPa - Example for surface pressure
def era5_sp(image):
    return image.divide(100).set('system:time_start', image.get('system:time_start'))

# Daily mean surface pressure
era5_sp = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                  .select('surface_pressure')
                  .filterBounds(fc)
                  .filter(ee.Filter.date(SDATE, EDATE))).map(era5_sp)


# Daily mean 10m u-component of wind
era5_u_wind_10m = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                          .select('u_component_of_wind_10m')
                          .filterBounds(fc)
                          .filter(ee.Filter.date(SDATE, EDATE)))



In [75]:
# Visualization palette for total precipitation
visTp = {
  'min': 0,
  'max': 0.1,
  'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

# Visualization palette for temperature (mean, min and max) and 2m dewpoint
# temperature
vis2mt = {
  'min': 250,
  'max': 320,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

# Visualization palette for u- and v-component of 10m wind
visWind = {
  'min': 0,
  'max': 30,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
}

# Visualization palette for pressure (surface pressure, mean sea level
# pressure) - adjust min and max values for mslp to min:990 and max:1050
visPressure = {
  'min': 500,
  'max': 1150,
  'palette': [
    '#01FFFF', '#058BFF', '#0600FF', '#DF00FF', '#FF00FF', '#FF8C00', '#FF8C00'
  ]
}


Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
Map

# Add layer to map
Map.addLayer(
    era5_tp.filter(ee.Filter.date('2019-07-15')), visTp,
    'Daily total precipitation sums')
Map.addLayer(
    era5_2d.filter(ee.Filter.date('2019-07-15')), vis2mt,
    'Daily mean 2m dewpoint temperature')
Map.addLayer(
    era5_2mt.filter(ee.Filter.date('2019-07-15')), vis2mt,
    'Daily mean 2m air temperature')
Map.addLayer(
    era5_u_wind_10m.filter(ee.Filter.date('2019-07-15')), visWind,
    'Daily mean 10m u-component of wind')
Map.addLayer(
    era5_sp.filter(ee.Filter.date('2019-07-15')), visPressure,
    'Daily mean surface pressure')

#Map.setCenter(21.2, 22.2, 2)

In [76]:
#tp_percentile_list

In [77]:
# total precipitation
# get area statistics
tp_mean_list = region_mean_ImageCollection(era5_tp)
tp_median_list = region_median_ImageCollection(era5_tp)
tp_stddev_list = region_stddev_ImageCollection(era5_tp)
tp_percentile_list = region_percentiles_ImageCollection(era5_tp)

# turn it into numpy array
tp_mean = newList(tp_mean_list, 'total_precipitation')
tp_median = newList(tp_median_list, 'total_precipitation')
tp_stddev = newList(tp_stddev_list, 'total_precipitation')
tp_percentile = p_list(tp_percentile_list, 'total_precipitation')

In [78]:
# 2m air temperature
# get area statistics
t2m_mean_list = region_mean_ImageCollection(era5_2mt)
t2m_median_list = region_median_ImageCollection(era5_2mt)
t2m_stddev_list = region_stddev_ImageCollection(era5_2mt)
t2m_percentile_list = region_percentiles_ImageCollection(era5_2mt)

# turn it into numpy array
t2m_mean = newList(t2m_mean_list, 'mean_2m_air_temperature')
t2m_median = newList(t2m_median_list, 'mean_2m_air_temperature')
t2m_stddev = newList(t2m_stddev_list, 'mean_2m_air_temperature')
t2m_percentile = p_list(t2m_percentile_list, 'mean_2m_air_temperature')

In [79]:
era5dates = ymdList(era5_tp)

## Export ERA5

In [80]:
# Export mean and stddev to file
site_era5 = pd.DataFrame()
site_era5['dates'] = era5dates
site_era5['precip_mean'] = tp_mean
site_era5['precip_stddev'] = tp_stddev
site_era5['t2m_mean'] = t2m_mean
site_era5['t2_stddev'] = t2m_stddev

filename = "/data/6ru/%s_era5.daily"%(sitename)
site_era5.to_csv(filename, index=False)

# ########################################
## GLDAS 2.1
# ########################################

In [ ]:
# GLDAS Net Longwave Radiation
gldas21_net_longwave_radiation = (ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
                          .select('Lwnet_tavg')
                          .filterBounds(fc)
                          .filter(ee.Filter.date('2016-01-01', '2016-12-31')))

# GLDAS Net Shortwave Radiation
gldas21_net_shortwave_radiation = (ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
                          .select('Swnet_tavg')
                          .filterBounds(fc)
                          .filter(ee.Filter.date('2016-01-01', '2016-12-31')))

#print(gldas21_net_longwave_radiation.size())
#print(gldas21_net_shortwave_radiation.size())


In [ ]:
## 3 hourly to daily time series -- take mean of values within the day
#1-day step
years = ee.List.sequence(2016, 2016)
period = ee.List.sequence(1, 30, 1)


def byYear(y):
    y = ee.Number(y)
    def dailymean(startDoy):
        startDoy = ee.Number(startDoy)
        return (gldas21_net_longwave_radiation
                .filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(startDoy, startDoy, 'day_of_year'))
                .median())
    return period.map(dailymean)

mapped_doy = years.map(byYear).flatten()
gldas21_net_longwave_radiation = ee.ImageCollection.fromImages(mapped_doy)


def byYear(y):
    y = ee.Number(y)
    def dailymean(startDoy):
        startDoy = ee.Number(startDoy)
        return (gldas21_net_shortwave_radiation
                .filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(startDoy, startDoy, 'day_of_year'))
                .median())
    return period.map(dailymean)

mapped_doy = years.map(byYear).flatten()
gldas21_net_shortwave_radiation = ee.ImageCollection.fromImages(mapped_doy)

In [ ]:
# net longwave radiation
# get area statistics
nlwr_mean_list = region_mean_ImageCollection(gldas21_net_longwave_radiation)
nlwr_median_list = region_median_ImageCollection(gldas21_net_longwave_radiation)
nlwr_stddev_list = region_stddev_ImageCollection(gldas21_net_longwave_radiation)
nlwr_percentile_list = region_percentiles_ImageCollection(gldas21_net_longwave_radiation)

# turn it into numpy array
nlwr_mean = newList(nlwr_mean_list, 'Lwnet_tavg')
nlwr_median = newList(nlwr_median_list, 'Lwnet_tavg')
nlwr_stddev = newList(nlwr_stddev_list, 'Lwnet_tavg')
nlwr_percentile = p_list(nlwr_percentile_list, 'Lwnet_tavg')



In [ ]:
# net shortwave radiation
# get area statistics
nswr_mean_list = region_mean_ImageCollection(gldas21_net_shortwave_radiation)
nswr_median_list = region_median_ImageCollection(gldas21_net_shortwave_radiation)
nswr_stddev_list = region_stddev_ImageCollection(gldas21_net_shortwave_radiation)
nswr_percentile_list = region_percentiles_ImageCollection(gldas21_net_shortwave_radiation)

# turn it into numpy array
nswr_mean = newList(nswr_mean_list, 'Swnet_tavg')
nswr_median = newList(nswr_median_list, 'Swnet_tavg')
nswr_stddev = newList(nswr_stddev_list, 'Swnet_tavg')
nswr_percentile = p_list(nswr_percentile_list, 'Swnet_tavg')

## Export GLDAS2.1

In [ ]:
# Export mean and stddev to file
site_gldas21 = pd.DataFrame()

site_gldas21['nlwrad_mean'] = nlwr_mean
site_gldas21['nlwrad_stddev'] = nlwr_stddev
site_gldas21['nswrad_mean'] = nswr_mean
site_gldas21['nswrad_stddev'] = nswr_stddev

filename = "/home/jbk/projects/climate/tropics/sentinel-2/data/gldas21/%s_gldas21.daily"%(sitename)
site_gldas21.to_csv(filename, index=False)

## Make few plots

In [ ]:
#Make lists with +/- SD
plus_error = [x + y for x,y in zip(tp_mean, tp_stddev)]
minus_error = [x - y for x,y in zip(tp_mean, tp_stddev)]

plt.axes([2, 0, 2, 1])
plt.grid(b = True, which = 'major', axis = 'x')
plt.fill_between(icdates, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
plt.plot(icdates, tp_mean, label='tp')

In [ ]:
def plot_mean_stddev(var_mean, var_stddev, var_name):
    #Make lists with +/- SD
    icdates=np.array(list(range(0,30)))
    plus_error = [x + y for x,y in zip(var_mean, var_stddev)]
    minus_error = [x - y for x,y in zip(var_mean, var_stddev)]

    plt.axes([2, 0, 2, 1])
    plt.grid(b = True, which = 'major', axis = 'x')
    plt.fill_between(icdates, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
    plt.plot(icdates, var_mean, label=var_name)
    plt.legend(loc = "upper right")

In [ ]:
plot_mean_stddev(tp_mean, tp_stddev, var_name="Total Precipitation")

In [ ]:
plot_mean_stddev(t2m_mean, t2m_stddev, var_name="2m Air Temperature")

In [ ]:
plot_mean_stddev(nlwr_mean, nlwr_stddev, var_name="Net Long Wave Radiation")

In [ ]:
plot_mean_stddev(nlwr_mean, nlwr_stddev, var_name="Net Short Wave Radiation")

In [ ]:
# Export mean and stddev to file
edata = np.column_stack((icdates, tp_mean, tp_stddev, t2m_mean, t2m_stddev))
site_era5 = pd.DataFrame()
site_era5['dates'] = icdates
site_era5['precip_mean'] = tp_mean
site_era5['precip_stddev'] = tp_stddev
site_era5['t2m_mean'] = t2m_mean
site_era5['t2_stddev'] = t2m_stddev
site_era5['nlwrad_mean'] = nlwr_mean
site_era5['nlwrad_stddev'] = nlwr_stddev
site_era5['nswrad_mean'] = nswr_mean
site_era5['nswrad_stddev'] = nswr_stddev

site_era5.to_csv('manaus_era5.daily', index=False)
#site_era5.to_csv('atto_era5.daily', index=False)
#site_era5.to_csv('corumba_era5.daily', index=False)
#site_era5.to_csv('mato_era5.daily', index=False)
#site_era5.to_csv('saop_era5.daily', index=False)

In [ ]:
# accumulate variables for Growing Degree Days calculation 
start_date = ee.Date('2016-01-01', 'UTC')
def maccum_byYear(y):
    def maccum_byMonth(m):
        return (ic
                .filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRage(m, m, 'month'))
                .
            )

In [ ]:
start_date = ee.Date('2016-01-01', 'UTC')
new_date = start_date.advance(2, 'month')
#print(start_date)
print(start_date, new_date)

In [ ]:
print(ee.Date)

In [ ]:
print(icdates)

In [ ]:
icdates

In [ ]:
gldasdates = ymdList(gldas21_net_longwave_radiation)

In [ ]:
print(gldasdates)